In [1]:
import pandas as pd

# Multi-scale-alignment

- Alignement of spatial resolution in modelsetup data
- Part of the energytransitionmodelling effort

@author: Mark Hupkens
@date: 16-05-2019

In [2]:
'''Import files'''
df_gem = pd.read_excel("D:/markhupkens/Dropbox/EnTransitionNL/0. Data/ModelSetUpEnergyNL01_MH.xlsx", sheet_name='gemeente')
df_buurt = pd.read_excel("D:/markhupkens/Dropbox/EnTransitionNL/0. Data/ModelSetUpEnergyNL01_MH.xlsx", sheet_name='buurt')
df_wijk = pd.read_excel("D:/markhupkens/Dropbox/EnTransitionNL/0. Data/ModelSetUpEnergyNL01_MH.xlsx", sheet_name='wijk')
df_mod = pd.read_excel("D:/markhupkens/Dropbox/EnTransitionNL/0. Data/ModelSetUpEnergyNL01_MH.xlsx", sheet_name='ModelSpecification')  

### Q: how do model scales line up in the data?

In [3]:
print(df_wijk.Entities.nunique())

3067


In [4]:
print(df_buurt.Mapping.nunique())

3067


A: There are more wijken included as a mapping on buurt-level, than as there are entities on wijk-level
A2: same applies for municipalities

## Allign data resolutions

In [5]:
'''Check and keep only rows if entities and mappings line up'''

# Keep only gemeenten that are mappings in wijk
df_gem_new = df_gem.loc[df_gem.Entities.isin(df_wijk.Mapping)==True]

# Keep only wijk rows whose mapping is an entity in gemeente
df_wijk_new = df_wijk.loc[df_wijk.Mapping.isin(df_gem_new.Entities)==True]

# Keep only wijk rows whose entity is a mapping in df_buurt
df_wijk_new = df_wijk_new.loc[df_wijk_new.Entities.isin(df_buurt.Mapping)==True]

# Keep only buurten whose mapping is an entity in df_wijk
df_buurt_new = df_buurt.loc[df_buurt.Mapping.isin(df_wijk_new.Entities)==True]

In [6]:
print('df_buurt_new has length',len(df_buurt_new))
print('df_wijk_new has length', len(df_wijk_new))
print('df_gem has length',len(df_gem_new))

df_buurt_new has length 11649
df_wijk_new has length 2729
df_gem has length 342


### Check new data resolutions

In [7]:
# buurt to wijk
(df_buurt_new.Mapping.isin(df_wijk_new.Entities)==True).value_counts()

True    11649
Name: Mapping, dtype: int64

In [8]:
# Wijk to gem
(df_wijk_new.Mapping.isin(df_gem_new.Entities)==True).value_counts()

True    2729
Name: Mapping, dtype: int64

In [9]:
# Gem to wijk
(df_gem_new.Entities.isin(df_wijk_new.Mapping)==True).value_counts()

True    342
Name: Entities, dtype: int64

All resolutions now line up, as all mappings in a low-resolution dataframe are included as entities in a high-resolution dataframe

### Data completeness

How much data has been lost in the alignment process

In [10]:
print(len(df_gem)-len(df_gem_new),'Gemeenten have been lost', (len(df_gem)-len(df_gem_new))/len(df_gem)*100 )
print(len(df_wijk)-len(df_wijk_new),'Wijken have been lost', (len(df_wijk)-len(df_wijk_new))/len(df_wijk)*100)
print(len(df_buurt)-len(df_buurt_new),'Buurten have been lost', (len(df_buurt)-len(df_buurt_new))/len(df_buurt)*100)

0 Gemeenten have been lost 0.0
338 Wijken have been lost 11.020541245516792
1339 Buurten have been lost 10.309516476747767


### Export data to new stup file

In [11]:
# export as xlsx to genereate new modelspecification file
from pandas import ExcelWriter

with pd.ExcelWriter('D:/markhupkens/Dropbox/EnTransitionNL/0. Data/ModelSetUpEnergyNL02EPMH3Aligned.xlsx') as writer:  # doctest: +SKIP
    df_buurt_new.to_excel(writer, sheet_name='buurt')
    df_wijk_new.to_excel(writer, sheet_name='wijk')
    df_gem_new.to_excel(writer, sheet_name='gemeente')
    df_mod.to_excel(writer, sheet_name='ModelSpecification')

In [12]:
print(len(df_buurt_new))
print(len(df_wijk_new))
print(len(df_gem_new))

11649
2729
342
